Video of Daylight of Volcano Camera Images
--

This focuses on a sub-area of the image and only images recorded during daylight.

Uses astroplan https://astroplan.readthedocs.io/en/latest/ to calculate sunrise and sunset times

In [72]:
import cv2
import matplotlib.pyplot as plt
import os
import datetime
import pytz
import pandas as pd

from astroplan import Observer
from astropy.time import Time
from astropy.coordinates import EarthLocation
import astropy.units as u

%matplotlib inline

In [73]:
def sunset(time):
    sunset = wi.sun_set_time(time, which="next") 
    sunset = wi.astropy_time_to_datetime(sunset)
    return (sunset)

In [74]:
def sunrise(time):
    sunrise = wi.sun_rise_time(time, which="next") 
    sunrise = wi.astropy_time_to_datetime(sunrise)
    return (sunrise)

In [76]:
def imfdt(imfile):
    #gets a datetime for an image file named like 20180101001002WIWR.jpg
    year = imfile[:4]
    month = imfile[4:6]
    day = imfile[6:8]
    hour = imfile[8:10]
    minute = imfile[10:12]
    second = imfile[12:14]
    dt = pytz.utc.localize(datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second)))
    return (dt)

In [102]:
def clahe(img):
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    #https://stackoverflow.com/questions/19363293/whats-the-fastest-way-to-increase-color-image-contrast-with-opencv-in-python-c
    global img2
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
    l, a, b = cv2.split(lab)  # split on 3 different channels

    l2 = clahe.apply(l)  # apply CLAHE to the L-channel

    lab = cv2.merge((l2,a,b))  # merge channels
    img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR

**Set up parameters for camera location**

In [77]:
#white island location, for sunrise and sunset
#http://astroplan.readthedocs.io/en/latest/api/astroplan.Observer.html#astroplan.Observer.sun_rise_time
white_island = EarthLocation(lat=-37.5245*u.deg, lon=177.1893*u.deg, height=0*u.m)
wi = Observer(location=white_island, name="WI", timezone="UTC")

**Set image date range and camera**

In [81]:
#UTC date range for images to view
date1 = '2018-03-01'
date2 = '2018-03-05'
camera = 'WIWR'

**Relevant directories**

In [84]:
#as mounted, this is image directory
camdir = '/run/user/1000/gvfs/ftp:host=ftp.geonet.org.nz/volcano_cams'

#directory to write crops
basedir = '/home/sherburn/geonet/volcam_images'
crpdir = os.path.join(basedir, 'crop')

**Loop through date folders**

In [104]:
for date in pd.date_range(date1, date2):
    year = date.year
    datestr = date.strftime('%Y%m%d')
    dateast = date.strftime('%Y-%m-%d')
    print (dateast)
    sun_set = sunset(Time(dateast))
    sun_rise = sunrise(Time(dateast))
    dydir = os.path.join(camdir, str(year), datestr, camera)
    
    for file in sorted(os.listdir(dydir)):
        if file.endswith('.jpg'):
            imfile = os.path.join(dydir, file)
            imtime = imfdt(file)
            if (imtime < sun_set or imtime > sun_rise):
                img = cv2.imread(imfile)
                crop = img[400:700,750:1450]
                clahe(crop) #enhanced the crop
                crfile = os.path.join(crpdir, file)
                cv2.imwrite(crfile, img2)
height,width,layers = crop.shape

2018-03-01
2018-03-02
2018-03-03
2018-03-04
2018-03-05


**Make video of cropped images**

In [105]:
vidname = camera+'_'+date1+'_'+date2+'.avi'
vidfile = os.path.join(crpdir, vidname)

fps = 6
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(vidfile,fourcc,fps,(width,height))
for file in sorted(os.listdir(crpdir)):
    if file.endswith('.jpg'):
        imfile = os.path.join(crpdir, file)
        img = cv2.imread(imfile)
        video.write(img)
cv2.destroyAllWindows
video.release()